## Import packages

In [1]:
from big_phoney import BigPhoney
import lyricsgenius as genius
import random
import nlpaug.augmenter.word as naw
import re
import numpy as np

#----------------For future live text generation-----------------
#import gpt_2_simple as gpt2

# model_name = "124M"
# model is saved into current directory under /models/124M/
# gpt2.download_gpt2(model_name=model_name) #need to run only once. comment out once done.

# Load models
#sess = gpt2.start_tf_sess()
#gpt2.load_gpt2(sess, run_name='run1')

#----------------------------------------------------------------

Using TensorFlow backend.


## Functions 

In [2]:
def get_target_lyrics(artist, title, api_key):
    
    api = genius.Genius(api_key)
    api.remove_section_headers = True # Removes section headers like "Verse" and "Intro"
    
    #get song lyrics
    song = api.search_song(title, artist)
    
    #clean up the target lyrics
    target = song.lyrics
    target = target.replace("\u2005"," ")
    target = target.replace("\\"," ")
    target = target.replace("\\n"," ")
    target = target.replace("("," ")
    target = target.replace(")"," ")
    target = target.replace("\n\n","\n")
    target = target.replace("\n\n\n","\n")
    target = target.replace("2x","")
    #create list of lines
    target_lyrics = target.split("\n")
    print("Target lyrics:")
    print(target_lyrics)
    return target_lyrics

In [4]:
get_target_lyrics('Idina Menzel','Let it go','Nc3VjqRNlEZiD9mqOQATDQ5PuZ4IjdgV13n7c39OIyNwsjuazZThQlQkdr_Hts4c')

Searching for "Let it go" by Idina Menzel...
Done.
Target lyrics:
['The snow glows white on the mountain tonight', 'Not a footprint to be seen', 'A kingdom of isolation', "And it looks like I'm the queen", 'The wind is howling like this swirling storm inside', "Couldn't keep it in, heaven knows I've tried", "Don't let them in, don't let them see", 'Be the good girl you always have to be', "Conceal, don't feel, don't let them know", 'Well, now they know', 'Let it go, let it go', "Can't hold it back anymore", 'Let it go, let it go', 'Turn away and slam the door', "I don't care what they're going to say", 'Let the storm rage on', 'The cold never bothered me anyway', 'Let it go, let it go', "Can't hold it back anymore", 'Let it go, let it go', 'Turn away and slam the door', 'Let it go  go, go, go go, go go, go go, go, go, go go ', 'Let it go', 'Let it go', 'Let it go', "It's funny how some distance makes everything seem small", "And the fears that once controlled me can't get to me at all"

['The snow glows white on the mountain tonight',
 'Not a footprint to be seen',
 'A kingdom of isolation',
 "And it looks like I'm the queen",
 'The wind is howling like this swirling storm inside',
 "Couldn't keep it in, heaven knows I've tried",
 "Don't let them in, don't let them see",
 'Be the good girl you always have to be',
 "Conceal, don't feel, don't let them know",
 'Well, now they know',
 'Let it go, let it go',
 "Can't hold it back anymore",
 'Let it go, let it go',
 'Turn away and slam the door',
 "I don't care what they're going to say",
 'Let the storm rage on',
 'The cold never bothered me anyway',
 'Let it go, let it go',
 "Can't hold it back anymore",
 'Let it go, let it go',
 'Turn away and slam the door',
 'Let it go  go, go, go go, go go, go go, go, go, go go ',
 'Let it go',
 'Let it go',
 'Let it go',
 "It's funny how some distance makes everything seem small",
 "And the fears that once controlled me can't get to me at all",
 "It's time to see what I can do",
 'T

In [ ]:
#----------------For future live text generation-----------------
#def generate_lyrics(): 
#    
#    gen_lyrics = gpt2.generate(sess, 
#                     length=250,
#                     temperature=0.9,
#                     include_prefix=False,
#                     #prefix="Deep in the night",
#                     nsamples=1,
#                     batch_size=1,
#                     return_as_list=True)[0]
#    
#    gen_lyrics = gen_lyrics.split("\n")
#    
#    return gen_lyrics
#----------------------------------------------------------------

In [7]:
def lyrics_to_list(file_name):
    with open(file_name, 'r') as file:
        data = file.read().split("====================")
        random_song = random.choice(data)
        target = random_song
        
        # clean generatedd lyrics
        target = target.replace("\u2005"," ")
        target = target.replace("\\"," ")
        target = target.replace("\\n"," ")
        target = target.replace("("," ")
        target = target.replace(")"," ")
        target = target.replace("\n\n","\n")
        target = target.replace("\n\n\n","\n")
        target = target.replace("2x","")
        gen_lyrics = target.split("\n")
        gen_lyrics = gen_lyrics[1:-1]
        print("Gen lyrics:")
        print(gen_lyrics)
    return gen_lyrics

#### The magical lyrics fitting function

This function takes lyrics generated by a fine-tuned GPT-2 model, and fits them to the syllabic structure of target lyrics (matches the number of syllables per line). 

In [83]:
def fit_lyrics(gen_lyrics, target_lyrics):
    
    print(gen_lyrics)
    
    aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="insert")
    print('aug')
    
    phoney = BigPhoney()
    print('initialized phoney')
    
    # Counts the number of syllables in each line
    def count_syls(text):
    
        schema = []
    
        for line in text:
            syls = phoney.count_syllables(line)
            schema.append(syls)
            #print(syls,line)
        
        return schema
    
    # Remove special characters and contractions from the line. This will make it easier to
    # augment lines that need augmentation. 
    def decontracted(phrase):
        # specific
        phrase = re.sub(r"won\'t", "will not", phrase)
        phrase = re.sub(r"can\'t", "can not", phrase)

        # general
        phrase = re.sub(r"n\'t", " not", phrase)
        phrase = re.sub(r"\'re", " are", phrase)
        phrase = re.sub(r"\'s", " is", phrase)
        phrase = re.sub(r"\'d", " would", phrase)
        phrase = re.sub(r"\'ll", " will", phrase)
        phrase = re.sub(r"\'t", " not", phrase)
        phrase = re.sub(r"\'ve", " have", phrase)
        phrase = re.sub(r"\'m", " am", phrase)
        phrase = re.sub('[!@#$?]', '', phrase)
        return phrase
    
    # Count the number of syllables in the generated and target texts.
    gen_schema = count_syls(gen_lyrics)
    target_schema = count_syls(target_lyrics)
#    print(target_schema)

    # make generated lyrics same length as target lyrics
    #target_len = len(target_schema)
    #del gen_schema[target_len:]
    #del gen_lyrics[target_len:]
    
    # initialize array for the new fitted lyrics
    new_lyrics = []
    
    # loop through each line and either find existing line to place into the current position, 
    # or augment the current line. 
    
    # keep track of repeated lines
    repeats = np.array([])
    for num, line in enumerate(gen_lyrics):
        print("line in gen_lyrics:")
        print(line)

        # if the line is already the right length, add it to the new lyrics. 
        if (gen_schema[num] == target_schema[num]): 
            new_lyrics.append(line)
            print("this line is good:")
            print(line)
        # if the line is not the right length, augment or delete
        elif (gen_schema[num] != target_schema[num]):
            line = decontracted(line)
            print("target syls:")
            print(target_schema[num])
            print("same line decontracted:")
            print(line)
            syls = gen_schema[num]
            # If we start with fewer syllables than we want, we augment. 
            while syls < target_schema[num]:
                print("not enough syls")
                original_line = line
                line = aug.augment(line)
                line = re.sub(r'[^\w\s]','',line)
                print(line)
                syls = phoney.count_syllables(line)
            #In case we overshoot (add too many syllables)
                if syls > target_schema[num]:
                    print("Oops we overshot")
                    print(line)
                    line = original_line
                    syls = phoney.count_syllables(line)
            new_line = line
#                print("the same line:")
            syls = gen_schema[num]
            words = line.split(" ")
            while syls > target_schema[num]:
                print("too many syls")
                original_words = words
                #instead of deleting the last word, try deleting a word randomly
                #del words[-1]
                words.pop(random.randrange(len(words))) 
                new_line = ' '.join(words)
                syls = phoney.count_syllables(new_line)
                print("after removing one:")
                print(new_line)
                #In case too many syllables are deleted
                if syls < target_schema[num]:
                    print("Oops, deleted too many")
                    print(line)
                    words = original_words
                    new_line = ' '.join(words)
                    syls = phoney.count_syllables(new_line)
                    print(syls)
                    print("the target was:")
                    print(target_schema[num])
                        
            new_lyrics.append(new_line)
                
    return new_lyrics


## Time to test everything

In [41]:
import sys
import pprint

In [22]:
artist = "Idina Menzel"
title = "Let it go"

In [50]:
lines = open('API.txt').readlines()
api_key = lines[0].strip()
target_lyrics = get_target_lyrics(artist, title, api_key)
gen_lyrics = lyrics_to_list('gpt2_gentext_slayer.txt')

Searching for "Let it go" by Idina Menzel...
Done.
Target lyrics:
['The snow glows white on the mountain tonight', 'Not a footprint to be seen', 'A kingdom of isolation', "And it looks like I'm the queen", 'The wind is howling like this swirling storm inside', "Couldn't keep it in, heaven knows I've tried", "Don't let them in, don't let them see", 'Be the good girl you always have to be', "Conceal, don't feel, don't let them know", 'Well, now they know', 'Let it go, let it go', "Can't hold it back anymore", 'Let it go, let it go', 'Turn away and slam the door', "I don't care what they're going to say", 'Let the storm rage on', 'The cold never bothered me anyway', 'Let it go, let it go', "Can't hold it back anymore", 'Let it go, let it go', 'Turn away and slam the door', 'Let it go  go, go, go go, go go, go go, go, go, go go ', 'Let it go', 'Let it go', 'Let it go', "It's funny how some distance makes everything seem small", "And the fears that once controlled me can't get to me at all"

In [84]:
new_lyrics = fit_lyrics(gen_lyrics, target_lyrics)

['an argument can bring you to your knees', 'you know you must be insane', 'to have witnessed such a dismal fail', 'murder, mayhem, anarchy', 'now that youve reached that critical mass', 'jealo bludgeoned to death', 'a lifeless object, a lifeless thing', 'drown in the gore, you must be insane', 'ive seen the atrocities done to you', 'inhumanities, atrophy, you must be insane', 'drown in the gore, these things must be true', 'you need to stop, you must be insane', 'you cannot stop, these things must be real', 'you need to stop, you must be insane', 'you can not stop, these things must be real', 'you can not stop, these things must be real', 'you can not stop, these things must be real', 'and that is when the pain begins', 'a blinding fear, returns', 'and paralyzes the unwary', 'stalking animal, ill control', 'screaming to be', 'life support for a dead animal', 'a lifeless object, no soul', 'bastard daughters raise the dead', 'indulge your sinful acts of terror', 'estranged from your cho

you drown in the gore  and you must be insane
Oops we overshot
you drown in the gore  and you must be insane
not enough syls
drown yourself in the gore  you absolutely must be insane
Oops we overshot
drown yourself in the gore  you absolutely must be insane
not enough syls
drown in the pink gore  you must be going insane
Oops we overshot
drown in the pink gore  you must be going insane
not enough syls
drown in the bloody gore  you absolutely must be insane
Oops we overshot
drown in the bloody gore  you absolutely must be insane
not enough syls
drown yourself in the gore  wherever you must be insane
Oops we overshot
drown yourself in the gore  wherever you must be insane
not enough syls
drown in the gore  you truly must be truly insane
Oops we overshot
drown in the gore  you truly must be truly insane
not enough syls
you drown in all the gore  you must be insane
Oops we overshot
you drown in all the gore  you must be insane
not enough syls
drown yourself in the black gore  you must be i

but you can not stop  for these things must not be real
Oops we overshot
but you can not stop  for these things must not be real
not enough syls
you can not stop  these things that must not be made real
Oops we overshot
you can not stop  these things that must not be made real
not enough syls
you can not quite stop  but these very things must be real
Oops we overshot
you can not quite stop  but these very things must be real
not enough syls
you really can not stop  these things  must be something real
Oops we overshot
you really can not stop  these things  must be something real
not enough syls
you can not quite stop  what these horrible things must be real
Oops we overshot
you can not quite stop  what these horrible things must be real
not enough syls
you can perhaps not stop  but these things must all be real
Oops we overshot
you can perhaps not stop  but these things must all be real
not enough syls
you can also not even stop  these ancient things must be real
Oops we overshot
you c

indulge in your sin withful acts of terror
not enough syls
never indul ge in destroying your sin withful acts of terror
Oops we overshot
never indul ge in destroying your sin withful acts of terror
not enough syls
 indul usge in your sin  withful acts of terror
not enough syls
indul usge in your worst sin withful of acts  of terror
Oops we overshot
indul usge in your worst sin withful of acts  of terror
not enough syls
indul usge more in your sin  withful acts of  terror
line in gen_lyrics:
estranged from your chosen race
target syls:
14
same line decontracted:
estranged from your chosen race
not enough syls
estranged from your chosen gypsy race
not enough syls
estranged  from your chosen gypsy race
not enough syls
estranged even from your chosen gypsy race
not enough syls
and estranged even then from your chosen gypsy race
not enough syls
and estranged even then  from your chosen gypsy  race
not enough syls
married and estranged even then from your chosen gypsy blood race
Oops we over

In [44]:
pprint.pprint(target_lyrics)

['The snow glows white on the mountain tonight',
 'Not a footprint to be seen',
 'A kingdom of isolation',
 "And it looks like I'm the queen",
 'The wind is howling like this swirling storm inside',
 "Couldn't keep it in, heaven knows I've tried",
 "Don't let them in, don't let them see",
 'Be the good girl you always have to be',
 "Conceal, don't feel, don't let them know",
 'Well, now they know',
 'Let it go, let it go',
 "Can't hold it back anymore",
 'Let it go, let it go',
 'Turn away and slam the door',
 "I don't care what they're going to say",
 'Let the storm rage on',
 'The cold never bothered me anyway',
 'Let it go, let it go',
 "Can't hold it back anymore",
 'Let it go, let it go',
 'Turn away and slam the door',
 'Let it go  go, go, go go, go go, go go, go, go, go go ',
 'Let it go',
 'Let it go',
 'Let it go',
 "It's funny how some distance makes everything seem small",
 "And the fears that once controlled me can't get to me at all",
 "It's time to see what I can do",
 'T

In [43]:
print(gen_lyrics)

['hello i love your ass', 'and just have me right here', 'ill take you down', 'and rip you up', 'and rip you right to right', 'you know ill never rest', 'condemned by a sea of blood', 'a physical fear', 'a mental sickness', 'i cant & wont treat', 'a sickness', 'like immunity', 'or is it the other way around', 'if you want the truth', 'i am the propaganda war machine', 'all of you cowards', 'are full of shit', 'and none of you fucking are', 'when i see you standing there', 'i fucking dont dont know what ive done', 'i just saw you bleed', 'when i saw you stood there', 'i just didnt know what ive done', 'i just saw you bleed', 'when i saw you stood there', 'i just didnt know what ive done', 'i just saw you stood there', 'i just didnt know what ive done', 'i just saw you stood there', 'i just didnt know what ive done', 'i just saw you stood there', 'i just didnt know what ive done', 'i just saw you stood there loud and clear', 'when i saw you standing there', 'i just didnt know what ive do

In [77]:
pprint.pprint(new_lyrics)

['an argument can bring you to your knees',
 'you know you must be insane',
 'to have witnessed a dismal fail',
 'murder, mayhem, anarchy',
 'now that way you ve reached to that critical mass',
 'je alo blu d geoned him to death',
 'a object, a lifeless thing',
 'drown in all the gore  you  must be insane',
 'ive the atrocities done to',
 'you insane',
 'drown the gore, these things must',
 'you need to stop, you insane',
 'you cannot stop, these things',
 'you need to stop, be insane',
 'you can not stop, these things must be real',
 'can not stop, things real',
 'you  can  not just stop  these things must be real',
 'is when the pain begins',
 'a blinding fear  will returns',
 'the unwary',
 'stalking animal, ill',
 'her little  tiny  screaming voice seemed to beg me not to be',
 'support a',
 'a no soul',
 'bastard the',
 'now ind ulge  your other sin ful acts out of terror',
 'estranged even less from your very own god chosen race',
 'im the one to watch  you both die',
 'lead the 

In [ ]:
# End tensorflow session 
sess.close()

In [ ]:
lines = open('TEST.txt').readlines()
api_key = lines[0].strip()

In [ ]:
print(api_key)

In [74]:
phoney = BigPhoney()
print('initialized phoney')

initialized phoney


In [80]:
new1 = phoney.count_syllables(new_lyrics[10])
new2 = phoney.count_syllables(new_lyrics[11])
new3 = phoney.count_syllables(new_lyrics[12])
print(new1,new2,new3)

6 7 6


In [81]:
target1 = phoney.count_syllables(target_lyrics[10])
target2 = phoney.count_syllables(target_lyrics[11])
target3 = phoney.count_syllables(target_lyrics[12])
print(target1,target2,target3)

6 7 6


In [67]:
print(target_lyrics[24])
print(new_lyrics[24])

Let it go
daughters


In [57]:
len(new_lyrics)

29

In [58]:
len(target_lyrics)

49